# The environment

The next three code cells contains commands that will configure your Coogle Colab environment. 

When you transfer the strategy to Quantiacs, remove these cells.

They are not relevant for the Quantiacs platform.

At first, setup the toolbox from github using pip:

In [ ]:
###DEBUG###

! pip install git+git://github.com/quantiacs/toolbox.git

# decrease height
from IPython.display import Javascript
display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 100})'))

Then install TA-Lib (indicators library) if you need it.

Instead of TA-Lib you can use qnt.ta or another library. In this case, skip the next cell.

Note that the installation can take several minutes.

In [ ]:
###DEBUG###
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
! make
!make install
!pip install Ta-Lib

# test import
import talib

# decrease height
from IPython.display import Javascript
display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 100})'))

Finally specify the api key and disable interactive charts.

You can find the api key in [your profile](https://quantiacs.com/personalpage/homepage).

We disable interactive charts in the library because interact+plotly do not work correctly in Google Colab.

In [3]:
###DEBUG###
import os

os.environ['API_KEY'] = "your api key here" 
os.environ['NONINTERACT'] = 'True' 

# The strategy

The next cell contains the strategy code itself.

In [ ]:
import xarray as xr

import qnt.stats as qnstats
import qnt.data as qndata
import qnt.output as qnout
import qnt.ta as qnta
import qnt.backtester as qnbt
import qnt.graph as qngraph


def strategy(data):
    # calc weights:
    close = data.sel(field="close")
    ma_slow = qnta.lwma(close, 250)
    ma_fast = qnta.lwma(close, 50)
    return xr.where(ma_fast > ma_slow, 1, -1)


# # SINGLE-PASS
# # ---
# # This is fast implementation, but it can easily become looking forward (common problem).
# # Use this approach for research and optimization. And use multi-pass to detect looking forward.
# data = qndata.futures.load_data(min_date="2005-01-01")  # load data

# output = strategy(data)
# output = qnout.clean(output, data) # fix common errors

# qnout.check(output, data) # check that weights are correct:
# qnout.write(output) # write results, necessary for submission:

# stats = qnstats.calc_stat(data, output.sel(time=slice("2006-01-01", None))) # calc stats
# print(stats.to_pandas().tail())
# # qngraph.make_major_plots(stats) # works in jupyter
# # ---


# MULTI-PASS
# ---
# Use this approach to make sure that your strategy is not looking forward.
weights = qnbt.backtest(
    competition_type="futures",  # Futures contest
    lookback_period=365,  # lookback in calendar days
    start_date="2006-01-01",
    strategy=strategy,
    analyze=True,
    build_plots=True
)

# ---

#Submit your strategy

Finally, you can transfer this notebook to Quantiacs.

Just create a new strategy notebook in [your area](https://quantiacs.com/personalpage/strategies) and upload this notebook to this new environment (replace `strategy.ipynb`).

Don't forget to **remove code cells from "The environment"** section.

If you have any special external dependency which is absent in Quantiacs jupyter environment, add their installation code to `init.ipynb`.

Then test and submit your strategy.